---
title: "CAS 570 Proposal"
author: "Nicole Brewer"
toc: false
number-sections: false
highlight-style: pygments
format:
  html: 
    code-fold: true
    html-math-method: katex
  pdf:
    geometry: 
      - top=30mm
      - left=30mm
jupyter: python3
---

In [1]:
class Publication():
    
    def __init__(self, id, doi, code_avail, documentation, platforms, sponsors):
        self.id = id
        self.doi = doi
        self.code_avail = code_avail
        self.documentation = documentation
        self.platforms = platforms
        self.sponsors = sponsors

In [ ]:
import openpyxl

# Open Workbook
wb = openpyxl.load_workbook(filename='Catalogdatabase-till2018b.xlsx', data_only=True)

In [ ]:
sheet = wb['publication']
ncol = sheet.max_column
nrow = sheet.max_row

In [ ]:
[(i, sheet.cell(row = 1, column = i).value) for i in range(1, ncol)]

In [ ]:
doc_i = (25, 34)
plat_i = (34, 85)
spon_i = (85, 136)
info_i = (1, 11, 24) # id, doi, code_avail

In [ ]:
document_types = [sheet.cell(row = 1, column = i).value for i in range(*doc_i)]
document_types = {i: s[s.find("(")+1:s.rfind(")")] for i, s in enumerate(document_types)}
platform_types = [sheet.cell(row = 1, column = i).value for i in range(*plat_i)]
platform_types = {i: s[s.find("(")+1:s.rfind(")")] for i, s in enumerate(platform_types)}
sponsor_types = [sheet.cell(row = 1, column = i).value for i in range(*spon_i)]
sponsor_types = {i: s[s.find("(")+1:s.rfind(")")] for i, s in enumerate(sponsor_types)}

In [ ]:
publications = []
for i in range(2, nrow + 1):
    info = [sheet.cell(row = i, column = j).value for j in info_i]
    info[2] = bool(info[2])
    d = [bool(sheet.cell(row = i, column = j).value) for j in range(*doc_i)]
    d = [document_types[i] for i, present in enumerate(d) if present]
    p = [bool(sheet.cell(row = i, column = j).value) for j in range(*plat_i)]
    p = [platform_types[i] for i, present in enumerate(p) if present]
    s = [bool(sheet.cell(row = i, column = j).value) for j in range(*spon_i)]
    s = [sponsor_types[i] for i, present in enumerate(s) if present]
    publications.append(Publication(*info, d, p, s))

In [ ]:
# use scopus to get citation data

In [ ]:
# add publications to network and add citation edges